In [1]:
import pandas as pd

from env import get_db_url

# 1 Attendance Data

Read the data from the attendance table and calculate an attendance percentage for each student. One half day is worth 50% of a full day, and 10 tardies is equal to one absence.

In [61]:
students = pd.read_sql('SELECT * FROM attendance', get_db_url('tidy_data'))
students = students.rename(columns = {'Unnamed: 0' : 'name'})
students

,name,2018-01-01,2018-01-02,2018-01-03,2018-01-04,2018-01-05,2018-01-06,2018-01-07,2018-01-08
0,Sally,P,T,T,H,P,A,T,T
1,Jane,A,P,T,T,T,T,A,T
2,Billy,A,T,A,A,H,T,P,T
3,John,P,T,H,P,P,T,P,P


In [63]:
students_tidy = students.melt(id_vars = 'name', var_name = 'date', value_name = 'attendance')
students_tidy.head()

,name,date,attendance
0,Sally,2018-01-01,P
1,Jane,2018-01-01,A
2,Billy,2018-01-01,A
3,John,2018-01-01,P
4,Sally,2018-01-02,T


In [68]:
students_tidy['attendance_value'] = students_tidy.attendance.map({'P' : 1, 'A' : 0, 'H' : 0.5, 'T' : 0.9})
students_tidy.head()

,name,date,attendance,attendance_value
0,Sally,2018-01-01,P,1.0
1,Jane,2018-01-01,A,0.0
2,Billy,2018-01-01,A,0.0
3,John,2018-01-01,P,1.0
4,Sally,2018-01-02,T,0.9


In [69]:
num_of_days = students_tidy.date.nunique()

students_tidy.groupby('name').attendance_value.sum() / num_of_days

name
Billy    0.5250
Jane     0.6875
John     0.9125
Sally    0.7625
Name: attendance_value, dtype: float64

# 2 Coffee Levels

- Read the coffee_levels table.

In [4]:
coffee_levels = pd.read_sql('SELECT * FROM coffee_levels;', get_db_url('tidy_data'))
coffee_levels.head()

,hour,coffee_carafe,coffee_amount
0,8,x,0.816164
1,9,x,0.451018
2,10,x,0.843279
3,11,x,0.335533
4,12,x,0.898291


In [22]:
coffee_levels.shape

(30, 3)

- Transform the data so that each carafe is in it's own column.

In [5]:
coffee_levels.coffee_carafe.value_counts()

x    10
y    10
z    10
Name: coffee_carafe, dtype: int64

In [20]:
coffee_levels_tidy = coffee_levels.pivot(index = 'hour', columns = 'coffee_carafe', values = 'coffee_amount')
coffee_levels_tidy

coffee_carafe,x,y,z
hour,,,
8,0.816164,0.189297,0.999264
9,0.451018,0.521502,0.915990
10,0.843279,0.023163,0.144928
11,0.335533,0.235529,0.311495
12,0.898291,0.017009,0.771947
13,0.310711,0.997464,0.398520
14,0.507288,0.058361,0.864464
15,0.215043,0.144644,0.436364
16,0.183891,0.544676,0.280621


In [21]:
coffee_levels_tidy = coffee_levels_tidy.reset_index()
coffee_levels_tidy.columns.name = ''
coffee_levels_tidy = coffee_levels_tidy.rename(columns = {'x' : 'carafe_x', 'y' : 'carafe_y', 'z' : 'carafe_z'})
coffee_levels_tidy

,hour,carafe_x,carafe_y,carafe_z
0,8,0.816164,0.189297,0.999264
1,9,0.451018,0.521502,0.915990
2,10,0.843279,0.023163,0.144928
3,11,0.335533,0.235529,0.311495
4,12,0.898291,0.017009,0.771947
5,13,0.310711,0.997464,0.398520
6,14,0.507288,0.058361,0.864464
7,15,0.215043,0.144644,0.436364
8,16,0.183891,0.544676,0.280621
9,17,0.391560,0.594126,0.436677


- Is this the best shape for the data?

This transformation does significantly reduce the number of rows in the dataframe. It also provides the data in a way that would be easy to visualize with a visualization tool. I believe this is a good shape for the data, however, this shape does violate the rule of not having one variable in multiple columns.

# 3 Cake Recipes

- Read the cake_recipes table. This data set contains cake tastiness scores for combinations of different recipes, oven rack positions, and oven temperatures.

In [25]:
cake_recipes = pd.read_sql('SELECT * FROM cake_recipes;', get_db_url('tidy_data'))
cake_recipes

,recipe:position,225,250,275,300
0,a:bottom,61.738655,53.912627,74.414730,98.786784
1,a:top,51.709751,52.009735,68.576858,50.228470
2,b:bottom,57.095320,61.904369,61.196980,99.248541
3,b:top,82.455004,95.224151,98.594881,58.169349
4,c:bottom,96.470207,52.001358,92.893227,65.473084
5,c:top,71.306308,82.795477,92.098049,53.960273
6,d:bottom,52.799753,58.670419,51.747686,56.183110
7,d:top,96.873178,76.101363,59.571620,50.971626


- Tidy the data as necessary.

> - The recipe:position columns needs to be separated into two columns.
> - The temperature columns need to be combined into one column.

In [89]:
cake_recipes_tidy = cake_recipes.melt(id_vars = 'recipe:position', var_name = 'temperature', value_name = 'score')
cake_recipes_tidy.head()

,recipe:position,temperature,score
0,a:bottom,225,61.738655
1,a:top,225,51.709751
2,b:bottom,225,57.095320
3,b:top,225,82.455004
4,c:bottom,225,96.470207


In [90]:
recipe_position = cake_recipes_tidy['recipe:position'].str.split(':', expand = True)
recipe_position.columns = ['recipe', 'position']
cake_recipes_tidy = pd.concat([recipe_position, cake_recipes_tidy], axis = 1)
cake_recipes_tidy = cake_recipes_tidy.drop(columns = ['recipe:position'])
cake_recipes_tidy.head()

,recipe,position,temperature,score
0,a,bottom,225,61.738655
1,a,top,225,51.709751
2,b,bottom,225,57.095320
3,b,top,225,82.455004
4,c,bottom,225,96.470207


- Which recipe, on average, is the best? recipe b

In [85]:
cake_recipes_tidy.groupby('recipe').score.mean().sort_values(ascending = False).head(1)

recipe
b    76.736074
Name: score, dtype: float64

- Which oven temperature, on average, produces the best results? 275

In [86]:
cake_recipes_tidy.groupby('temperature').score.mean().sort_values(ascending = False).head(1)

temperature
275    74.886754
Name: score, dtype: float64

- Which combination of recipe, rack position, and temperature gives the best result? recipe b, bottom rack, 300 degrees

In [87]:
columns = ['recipe', 'position', 'temperature']
cake_recipes_tidy.groupby(columns).score.mean().sort_values(ascending = False).head(1)

recipe  position  temperature
b       bottom    300            99.248541
Name: score, dtype: float64

# 4 Bonus

## Melted Table

In [71]:
melted = pd.read_sql('SELECT * FROM melted;', get_db_url('tidy_data'))
melted

,date,variable,value
0,2000-01-03,A,-0.456694
1,2000-01-03,B,-0.446287
2,2000-01-03,C,-0.839698
3,2000-01-03,D,0.963339
4,2000-01-04,A,-1.758507
...,...,...,...
115,2000-02-10,D,-1.049402
116,2000-02-11,A,-0.171207
117,2000-02-11,B,-1.554717
118,2000-02-11,C,-0.950767


In [72]:
melted.variable.value_counts()

A    30
B    30
C    30
D    30
Name: variable, dtype: int64

In [76]:
# We'll separate the variable column into 4 columns

melted_tidy = melted.pivot(index = 'date', columns = 'variable', values = 'value')
melted_tidy = melted_tidy.reset_index()
melted_tidy.columns.name = ''
melted_tidy.head()

,date,A,B,C,D
0,2000-01-03,-0.456694,-0.446287,-0.839698,0.963339
1,2000-01-04,-1.758507,-0.585874,1.824410,0.247688
2,2000-01-05,-1.233451,0.483213,0.928141,-0.547720
3,2000-01-06,1.085606,-0.483495,-1.897256,0.016627
4,2000-01-07,2.219914,1.195858,-0.506973,-0.498261


## Weather Table

In [77]:
weather = pd.read_sql('SELECT * FROM weather;', get_db_url('tidy_data'))
weather.head()

,id,year,month,element,d1,d2,d3,d4,d5,d6,...,d22,d23,d24,d25,d26,d27,d28,d29,d30,d31
0,MX17004,2010,1,tmax,NaN,NaN,NaN,NaN,NaN,NaN,...,None,NaN,None,NaN,NaN,NaN,NaN,NaN,27.8,NaN
1,MX17004,2010,1,tmin,NaN,NaN,NaN,NaN,NaN,NaN,...,None,NaN,None,NaN,NaN,NaN,NaN,NaN,14.5,NaN
2,MX17004,2010,2,tmax,NaN,27.3,24.1,NaN,NaN,NaN,...,None,29.9,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,MX17004,2010,2,tmin,NaN,14.4,14.4,NaN,NaN,NaN,...,None,10.7,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,MX17004,2010,3,tmax,NaN,NaN,NaN,NaN,32.1,NaN,...,None,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [93]:
# We have too many columns so we will collapse the day columns into a single column
weather_tidy = weather.melt(id_vars = ['id', 'year', 'month', 'element'], var_name = 'day')
weather_tidy.head()

,id,year,month,element,day,value
0,MX17004,2010,1,tmax,d1,NaN
1,MX17004,2010,1,tmin,d1,NaN
2,MX17004,2010,2,tmax,d1,NaN
3,MX17004,2010,2,tmin,d1,NaN
4,MX17004,2010,3,tmax,d1,NaN
